In [1]:
import os
import re
import glob
import math
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import scienceplots  # pip install SciencePlots

# Use SciencePlots + LaTeX
plt.style.use(['science', 'ieee'])  # or ['science', 'nature'] if you prefer
mpl.rcParams.update({
    "text.usetex": True,                 # requires a LaTeX distribution
    "font.family": "serif",
    "font.serif": ["Computer Modern"],   # classic LaTeX look
    "axes.unicode_minus": False,         # make sure '-' renders with LaTeX
    "text.latex.preamble": r"\usepackage{amsmath}",  # keep preamble minimal
})



# Example layouts handled:
#  - runs/served/l050_m100_s11/summary.csv
#  - runs/time/l050_m100_s11/summary.csv
#  - runs/time/T100000/l050_m100_s11/summary.csv   (T is optional directory)
served_dir_pat = re.compile(
    r".*/runs/served/l(?P<l>\d{3})_m(?P<m>\d{3})_s(?P<s>\d+)/summary\.csv$"
)
time_dir_pat = re.compile(
    r".*/runs/time/(?:T(?P<T>\d+)/)?l(?P<l>\d{3})_m(?P<m>\d{3})_s(?P<s>\d+)/summary\.csv$"
)

METRICS = [
    ('AvgQ(Lq)',         'Lq(theory)',   r'Average queue length $L_q$', 'Lq'),
    ('AvgDelay(W)',      'W(theory)',    r'Average system time $W$',    'W'),
    ('AvgWait(Wq)',      'Wq(theory)',   r'Average wait time $W_q$',    'Wq'),
    ('Utilization(rho)', 'rho(theory)',  r'Utilization $\rho$',         'rho'),
]


def parse_path(path: str):
    m = served_dir_pat.match(path)
    if m:
        g = m.groupdict()
        return {
            'sweep'  : 'served',
            'lambda' : int(g['l']) / 100.0,
            'mu'     : int(g['m']) / 100.0,
            'T'      : None,
            'seed'   : int(g['s']),
        }
    m = time_dir_pat.match(path)
    if m:
        g = m.groupdict()
        T = g.get('T')
        return {
            'sweep'  : 'time',
            'lambda' : int(g['l']) / 100.0,
            'mu'     : int(g['m']) / 100.0,
            'T'      : int(T) if T is not None else None,
            'seed'   : int(g['s']),
        }
    return None

def read_summary(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df.columns = [str(c).strip() for c in df.columns]
    return df

def _safe_float(x):
    try:
        return float(x)
    except Exception:
        return None

def metric_values(df: pd.DataFrame, empirical_name: str, theory_name: str):
    mean = ci_low = ci_up = theory = None

    # empirical row
    emp = df[df['Metric'] == empirical_name]
    if not emp.empty:
        row = emp.iloc[0]
        mean = _safe_float(row.get('Mean'))
        # CI columns may be missing or NaN
        ci_low = _safe_float(row.get('CI_Lower')) if 'CI_Lower' in df.columns else None
        ci_up  = _safe_float(row.get('CI_Upper')) if 'CI_Upper' in df.columns else None

    # theory row
    th = df[df['Metric'] == theory_name]
    if not th.empty:
        rowt = th.iloc[0]
        # some summary.csv files put theory under 'Mean', others may leave blanks -> handle both
        theory = _safe_float(rowt.get('Mean'))

    return mean, ci_low, ci_up, theory

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def plot_sweep(points, x_getter, x_label, title_prefix, name_suffix, save_dir):
    pts = sorted(points, key=x_getter)
    outfiles = []

    for emp_name, th_name, y_label, short_key in METRICS:
        xs, ys, yerr_low, yerr_up, thys = [], [], [], [], []

        for e in pts:
            df = e['df']
            mean, ci_low, ci_up, theory = metric_values(df, emp_name, th_name)
            if mean is None or (isinstance(mean, float) and np.isnan(mean)):
                continue

            x = x_getter(e)
            xs.append(x)
            ys.append(mean)
            if ci_low is not None and ci_up is not None:
                yerr_low.append(abs(mean - ci_low))
                yerr_up.append(abs(ci_up - mean))
            else:
                yerr_low.append(0.0); yerr_up.append(0.0)
            thys.append(theory)

        # ⬇️ Skip one-point “curves”
        if len(set(xs)) < 2:
            continue

        plt.figure(figsize=(4, 3))
        yerr = np.vstack([yerr_low, yerr_up])
        plt.errorbar(xs, ys, yerr=yerr, fmt='o', capsize=4, label='Simulation (± CI)', color='blue')
        th_pairs = [(x, t) for x, t in zip(xs, thys) if t is not None and not (isinstance(t, float) and np.isnan(t))]
        if th_pairs:
            tx, ty = zip(*th_pairs)
            plt.scatter(tx, ty, marker='*', s=150, label='Theory', color='red')

        plt.xlabel(x_label); plt.ylabel(y_label)
        plt.title(f"{title_prefix} — {y_label}")
        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.6)
        plt.legend()

        os.makedirs(save_dir, exist_ok=True)
        fname = f"{name_suffix}_{short_key}.png"
        fpath = os.path.join(save_dir, fname)
        plt.tight_layout(); plt.savefig(fpath, dpi=150); plt.close()
        outfiles.append(fpath)

    return outfiles


def discover_and_plot(base_dir=".", save_dir="plots"):
    all_csvs = glob.glob(os.path.join(base_dir, "runs", "**", "summary.csv"), recursive=True)
    if not all_csvs:
        print("No summary.csv files found under", os.path.join(base_dir, "runs"))
        return []

    entries = []
    for p in sorted(all_csvs):
        meta = parse_path(p)
        if meta is None:
            continue
        try:
            df = read_summary(p)
        except Exception as e:
            print("Skipping unreadable:", p, e)
            continue
        e = dict(meta); e['csv_path'] = p; e['df'] = df
        entries.append(e)

    if not entries:
        print("Found summary.csv files, but none matched expected folder pattern.")
        return []

    # pick lowest-seed per (sweep, λ, μ, T)
    keyed = {}
    for e in entries:
        key = (e['sweep'], e['lambda'], e['mu'], e.get('T'))
        if key not in keyed or e['seed'] < keyed[key]['seed']:
            keyed[key] = e
    chosen = list(keyed.values())

    # ===== SERVED groups =====
    served_var_lambda = {}  # vary λ at fixed μ
    served_var_mu     = {}  # vary μ at fixed λ
    for e in chosen:
        if e['sweep'] == 'served':
            served_var_lambda.setdefault(e['mu'], []).append(e)
            served_var_mu.setdefault(e['lambda'], []).append(e)

    # keep only groups with >1 distinct x
    served_var_lambda = {
        mu: pts for mu, pts in served_var_lambda.items()
        if len({p['lambda'] for p in pts}) > 1
    }
    served_var_mu = {
        lam: pts for lam, pts in served_var_mu.items()
        if len({p['mu'] for p in pts}) > 1
    }

    generated = []

    # Served: vary λ at fixed μ
    for mu_val, points in served_var_lambda.items():
        uniq = {}
        for e in points:
            key = e['lambda']
            if key not in uniq or e['seed'] < uniq[key]['seed']:
                uniq[key] = e
        used = list(uniq.values())
        generated += plot_sweep(
            used,
            x_getter=lambda e: e['lambda'],
            x_label=r"Arrival rate $\lambda$",
            title_prefix=rf"BY_SERVED sweep ($\mu$ = {mu_val:.2f})",
            name_suffix=f"served_var_lambda_m{int(round(mu_val*100)):03d}",
            save_dir=save_dir
        )

    # Served: vary μ at fixed λ
    for lam_val, points in served_var_mu.items():
        uniq = {}
        for e in points:
            key = e['mu']
            if key not in uniq or e['seed'] < uniq[key]['seed']:
                uniq[key] = e
        used = list(uniq.values())
        generated += plot_sweep(
            used,
            x_getter=lambda e: e['mu'],
            x_label=r"Service rate $\mu$",
            title_prefix=rf"BY_SERVED sweep ($\lambda$ = {lam_val:.2f})",
            name_suffix=f"served_var_mu_l{int(round(lam_val*100)):03d}",
            save_dir=save_dir
        )


    # ===== TIME groups (already had both; keep filters) =====
    time_groups_var_mu = {}
    time_groups_var_lambda = {}
    for e in chosen:
        if e['sweep'] == 'time':
            time_groups_var_mu.setdefault((e['lambda'], e.get('T')), []).append(e)
            time_groups_var_lambda.setdefault((e['mu'], e.get('T')), []).append(e)

    def _filter_multi(points, keyfunc):
        return len({keyfunc(x) for x in points}) > 1

    time_groups_var_mu = {k: v for k, v in time_groups_var_mu.items()
                          if _filter_multi(v, keyfunc=lambda x: x['mu'])}
    time_groups_var_lambda = {k: v for k, v in time_groups_var_lambda.items()
                              if _filter_multi(v, keyfunc=lambda x: x['lambda'])}

    # Time: vary μ at fixed λ (and fixed T)
    for (lam_val, T_val), points in time_groups_var_mu.items():
        uniq = {}
        for e in points:
            key = e['mu']
            if key not in uniq or e['seed'] < uniq[key]['seed']:
                uniq[key] = e
        used = list(uniq.values())
        ttag = f"_T{T_val}" if T_val is not None else ""
        generated += plot_sweep(
            used,
            x_getter=lambda e: e['mu'],
            x_label=r"Service rate $\mu$",
            title_prefix=rf"BY_TIME sweep ($\lambda$ = {lam_val:.2f}"
                        + (rf", $T$ = {T_val}" if T_val is not None else "") + ")",
            name_suffix=f"time_var_mu_l{int(round(lam_val*100)):03d}{ttag}",
            save_dir=save_dir
        )


    # Time: vary λ at fixed μ (and fixed T)
    for (mu_val, T_val), points in time_groups_var_lambda.items():
        uniq = {}
        for e in points:
            key = e['lambda']
            if key not in uniq or e['seed'] < uniq[key]['seed']:
                uniq[key] = e
        used = list(uniq.values())
        ttag = f"_T{T_val}" if T_val is not None else ""
        generated += plot_sweep(
            used,
            x_getter=lambda e: e['lambda'],
            x_label=r"Arrival rate $\lambda$",
            title_prefix=rf"BY_TIME sweep ($\mu$ = {mu_val:.2f}"
                        + (rf", $T$ = {T_val}" if T_val is not None else "") + ")",
            name_suffix=f"time_var_lambda_m{int(round(mu_val*100)):03d}{ttag}",
            save_dir=save_dir
        )


    print(("Saved " + str(len(generated)) + " plot(s) to " + save_dir)
          if generated else
          "No sweep groups with >1 points were found. Check your runs directory structure.")
    return generated


In [2]:
# ap = argparse.ArgumentParser(description="Plot DES sweeps from runs/**/summary.csv")
# ap.add_argument("--base-dir", default=".", help="Project base directory containing 'runs/'")
# ap.add_argument("--save-dir", default="plots", help="Directory to save PNG plots")
# args = ap.parse_args()

save_dir = "plots"
base_dir = "."

discover_and_plot(base_dir=base_dir, save_dir=save_dir)

Saved 16 plot(s) to plots


['plots/served_var_lambda_m110_Lq.png',
 'plots/served_var_lambda_m110_W.png',
 'plots/served_var_lambda_m110_Wq.png',
 'plots/served_var_lambda_m110_rho.png',
 'plots/served_var_mu_l070_Lq.png',
 'plots/served_var_mu_l070_W.png',
 'plots/served_var_mu_l070_Wq.png',
 'plots/served_var_mu_l070_rho.png',
 'plots/time_var_mu_l070_Lq.png',
 'plots/time_var_mu_l070_W.png',
 'plots/time_var_mu_l070_Wq.png',
 'plots/time_var_mu_l070_rho.png',
 'plots/time_var_lambda_m110_Lq.png',
 'plots/time_var_lambda_m110_W.png',
 'plots/time_var_lambda_m110_Wq.png',
 'plots/time_var_lambda_m110_rho.png']